In [1]:
import sqlite3
import pandas as pd

# Create a database connection
conn = sqlite3.connect(":memory:")  # Creates a temporary in-memory database
cursor = conn.cursor()

# Check if SQLite is working
cursor.execute("SELECT sqlite_version();")
print("SQLite is working! Version:", cursor.fetchone()[0])

SQLite is working! Version: 3.45.3


In [2]:
# Create employees table
cursor.execute('''
CREATE TABLE employees (
    id INTEGER PRIMARY KEY,
    name TEXT,
    salary INTEGER,
    department TEXT
);
''')

In [3]:
cursor.executemany('''
INSERT INTO employees (name, salary, department) VALUES (?, ?, ?);
''', [
    ('Alice', 60000, 'HR'),
    ('Bob', 50000, 'IT'),
    ('Charlie', 55000, 'Finance'),
    ('David', 75000, 'IT'),
    ('Eva', 62000, 'HR'),
    ('Frank', 70000, 'Finance'),
    ('Grace', 80000, 'IT')
])
conn.commit()
print("Table created & data inserted successfully!")

Table created & data inserted successfully!


In [4]:
# Create departments table
cursor.execute('''
CREATE TABLE departments (
    department TEXT PRIMARY KEY,
    location TEXT
);
''')

In [5]:
# Insert department data
cursor.executemany('''
INSERT INTO departments (department, location) VALUES (?, ?);
''', [
    ('HR', 'New York'),
    ('IT', 'San Francisco'),
    ('Finance', 'Chicago'),
    ('Marketing', 'Los Angeles')
])

conn.commit()
print("Departments table created & data inserted successfully!")

Departments table created & data inserted successfully!


In [14]:
query = '''
SELECT name, salary, department
FROM employees
WHERE salary > (SELECT AVG(Salary) from employees);
'''
df = pd.read_sql(query, conn)
df


,name,salary,department
0,David,75000,IT
1,Frank,70000,Finance
2,Grace,80000,IT


In [15]:
query = '''
SELECT AVG(salary) FROM employees GROUP BY department;
'''
df = pd.read_sql(query, conn)
df

,AVG(salary)
0,62500.000000
1,61000.000000
2,68333.333333


In [20]:
query = '''
SELECT e.name, e.salary, e.department, d.avg_salary AS Average_Department_Salary
FROM employees e
JOIN (SELECT department, AVG(salary) AS avg_salary FROM employees GROUP BY department) d ON e.department=d.department;
'''
df = pd.read_sql(query, conn)
df

,name,salary,department,Average_Department_Salary
0,Alice,60000,HR,61000.000000
1,Bob,50000,IT,68333.333333
2,Charlie,55000,Finance,62500.000000
3,David,75000,IT,68333.333333
4,Eva,62000,HR,61000.000000
5,Frank,70000,Finance,62500.000000
6,Grace,80000,IT,68333.333333
